Тестовое задание. Кутузов Артём (Телеграмм @TemaCu)

In [173]:
import json
from copy import deepcopy

import pandas as pd

# Загрузка и просмотр данных

In [174]:
# Загрузим json данные в виде слованя Python
with open("./trial_task.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [175]:
# На этом шаге развернём вложенную структуру исходных данных
# в развёрнутую структуру, где каждая строчка будущего датафрейма
# будет представлять из себя данные о заказе, а так же данные о конкретном (одном) товаре
list_of_expanded_orders = []

for order in data:
    order_copy = deepcopy(order)
    products = order_copy.get("products")
    order_copy.pop("products")

    for product in products:
        list_of_expanded_orders.append(order_copy | product)

In [176]:
dataframe = pd.DataFrame(list_of_expanded_orders)
dataframe.head(10)

,order_id,warehouse_name,highway_cost,product,price,quantity
0,11973,Мордор,-70,ломтик июльского неба,450,1
1,11973,Мордор,-70,билет в Израиль,1000,3
2,11973,Мордор,-70,статуэтка Ленина,200,3
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1
4,85794,отель Лето,-50,зеленая пластинка,10,2
5,33684,Мордор,-30,билет в Израиль,1000,2
6,33684,Мордор,-30,зеленая пластинка,10,1
7,25824,отель Лето,-75,автограф Стаса Барецкого,600,1
8,25824,отель Лето,-75,статуэтка Ленина,200,1
9,25824,отель Лето,-75,плюмбус,250,1


# Тариф стоимости доставки для каждого склада

In [177]:
# Так как у каждого склада неизменный тариф стоимости доставки, выведеи уникальные значения
# для названий складов и соответственно их значения стоимости доставки
# P.S. важно обратить внимание, что в задании не указано найти сумму или иную величину стоимости доставки,
# исходя из чего игнорируются любые манипуляции с данными
df_where_warehouse_name_is_unique = dataframe.drop_duplicates(subset=["warehouse_name"])
df_where_warehouse_name_is_unique[["warehouse_name", "highway_cost"]]

,warehouse_name,highway_cost
0,Мордор,-70
3,хутор близ Диканьки,-15
4,отель Лето,-50
10,остров невезения,-15
11,гиперборея,-160


# Узнаем количество, доход, расход и приюыль для каждого УНИКАЛЬНОГО товара

In [178]:
def agg_func(row):
    """ Аггрегирующая функция. """
    product_count = sum(row["quantity"])
    incomes = sum(row["price"] * product_count)
    expenses = sum(row.groupby("warehouse_name").apply(lambda x: x["highway_cost"] * x["quantity"]))

    series = pd.Series(
        [
            product_count,
            incomes,
            expenses,
            incomes - abs(expenses),
        ],
        index=["quantity", "income", "expenses", "profit"]
    )

    return series

dataframe.groupby("product").apply(agg_func)

,quantity,income,expenses,profit
product,,,,
автограф Стаса Барецкого,48,777600,-3930,773670
билет в Израиль,58,1682000,-5110,1676890
зеленая пластинка,61,18910,-4595,14315
ломтик июльского неба,42,396900,-2495,394405
плюмбус,65,536250,-4655,531595
подписка на suppi-блог,33,94050,-2670,91380
статуэтка Ленина,68,503200,-4565,498635


# Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

In [179]:
def agg_func(row):
    """ Аггрегирующая функция. """
    product_count = sum(row["quantity"])
    incomes = sum(row["price"] * product_count)
    expenses = sum(row["highway_cost"] * row["quantity"])

    series = pd.Series(
        [
            incomes - abs(expenses),
        ],
        index=["order_profit"]
    )

    return series

order_profit_df = dataframe.groupby("order_id").apply(agg_func)

print(order_profit_df["order_profit"].mean())
order_profit_df.head()

2902.9


,order_profit
order_id,
124,615
1391,480
2091,2400
2108,400
2558,1675


# Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

In [180]:
result = dataframe.copy()
result["profit"] = (result["price"] * result["quantity"]) - abs(result["highway_cost"] * result["quantity"])

result = result.join(
    pd.DataFrame(
        result.groupby("warehouse_name").apply(
            lambda x: sum((x["price"] * x["quantity"]) - abs(x["highway_cost"] * x["quantity"]))
        ),
        columns=["profit_warehouse"]
    ),
    how='inner',
    on='warehouse_name'
)


result["percent_profit_product_of_warehouse"] = round((result["profit"] / result["profit_warehouse"]) * 100, 2)

result[['warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse']]


,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,ломтик июльского неба,1,380,2.51
1,Мордор,билет в Израиль,3,2790,18.42
2,Мордор,статуэтка Ленина,3,390,2.57
5,Мордор,билет в Израиль,2,1940,12.81
6,Мордор,зеленая пластинка,1,-20,-0.13
...,...,...,...,...,...
165,гиперборея,ломтик июльского неба,2,780,2.51
166,гиперборея,подписка на suppi-блог,2,60,0.19
167,гиперборея,статуэтка Ленина,2,160,0.51
168,гиперборея,зеленая пластинка,2,-220,-0.71


# Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться
'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [181]:
result = result.sort_values("percent_profit_product_of_warehouse")
result["accumulated_percent_profit_product_of_warehouse"] = result["percent_profit_product_of_warehouse"].cumsum()
result

,order_id,warehouse_name,highway_cost,product,price,quantity,profit,profit_warehouse,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
118,35330,отель Лето,-125,зеленая пластинка,10,3,-345,21950,-1.57,-1.57
174,83889,отель Лето,-150,зеленая пластинка,10,2,-280,21950,-1.28,-2.85
62,32313,хутор близ Диканьки,-120,зеленая пластинка,10,3,-330,30250,-1.09,-3.94
158,6535,хутор близ Диканьки,-105,зеленая пластинка,10,3,-285,30250,-0.94,-4.88
20,64013,хутор близ Диканьки,-105,зеленая пластинка,10,3,-285,30250,-0.94,-5.82
...,...,...,...,...,...,...,...,...,...,...
136,59590,отель Лето,-75,билет в Израиль,1000,3,2775,21950,12.64,442.94
5,33684,Мордор,-30,билет в Израиль,1000,2,1940,15150,12.81,455.75
64,56905,Мордор,-20,билет в Израиль,1000,2,1960,15150,12.94,468.69
68,16240,Мордор,-20,билет в Израиль,1000,2,1960,15150,12.94,481.63


# Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A.
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

In [182]:
def apply_fn(x):
    """ Разделение на категории. """

    if x <= 70:

        return "A"
    
    elif x <= 90:

        return "B"

    else:
        return "C"

result["category"] = result["accumulated_percent_profit_product_of_warehouse"].apply(apply_fn)
result

,order_id,warehouse_name,highway_cost,product,price,quantity,profit,profit_warehouse,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
118,35330,отель Лето,-125,зеленая пластинка,10,3,-345,21950,-1.57,-1.57,A
174,83889,отель Лето,-150,зеленая пластинка,10,2,-280,21950,-1.28,-2.85,A
62,32313,хутор близ Диканьки,-120,зеленая пластинка,10,3,-330,30250,-1.09,-3.94,A
158,6535,хутор близ Диканьки,-105,зеленая пластинка,10,3,-285,30250,-0.94,-4.88,A
20,64013,хутор близ Диканьки,-105,зеленая пластинка,10,3,-285,30250,-0.94,-5.82,A
...,...,...,...,...,...,...,...,...,...,...,...
136,59590,отель Лето,-75,билет в Израиль,1000,3,2775,21950,12.64,442.94,C
5,33684,Мордор,-30,билет в Израиль,1000,2,1940,15150,12.81,455.75,C
64,56905,Мордор,-20,билет в Израиль,1000,2,1960,15150,12.94,468.69,C
68,16240,Мордор,-20,билет в Израиль,1000,2,1960,15150,12.94,481.63,C
